In [1]:

import os, sys, math, time
import numpy as np
import numpy.linalg as la
import plotly.graph_objects as go
import plotly.express as ex
from plotly.subplots import make_subplots
import pandas as pd

import json as js
import _pickle as pickle
import bz2
import ray

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
from collections import OrderedDict

from ray import tune
from ray.tune.suggest.bayesopt import BayesOptSearch
import shutil
import tempfile
from ray.tune import CLIReporter, JupyterNotebookReporter
from ray.tune.schedulers import ASHAScheduler, PopulationBasedTraining
from ray.tune.integration.pytorch_lightning import TuneReportCallback, \
    TuneReportCheckpointCallback

import pytorch_lightning as pl
from pytorch_lightning.utilities.cloud_io import load as pl_load
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import EarlyStopping

from cytoolz import sliding_window
sys.path.append("../")
import func

## Simple MLP Autoencoder
$
f(x,\theta) = dec(enc(x,\theta_1), \theta_2) = x,   \quad \theta = (\theta_1, \theta_2)
$

$
enc(x, \theta_1) = z, \quad   z \in Z \quad \text{ = latent space}
$

$
dec(z, \theta_2) = x, \quad   x \in X \quad \text{ = input space}
$

This model uses simple Multi-layered perceptron (MLP) for both encoder and decoder.

$
enc = dec = mlp(X, \theta), \quad \theta = W, b

$
mlp(X, W) = f(f(X \cdot w_1 + b_1) \cdot w_2 + b_2) \cdot w_3 + b_3
$



In [17]:
# Test torch lightning + ray tune

class MLP(pl.LightningModule):
    def __init__(self, config:dict=None, dimensions:list=None, loss_fn=None,
                 dataset=None, train_set=None, val_set=None, test_set=None,
                 keep_prob:float=.2, name:str="model", load=False):

        super(MLP, self).__init__()
        self.name = name
        self.dimensions = dimensions
        self.keep_prob = keep_prob
        if load:
            self.build()
        else:
            self.k = config["k"]
            self.learning_rate = config["lr"]
            dimensions.append(self.k)
            self.dimensions = dimensions
            self.loss_fn = loss_fn
            self.keep_prob = keep_prob          #   %
            self.batch_size = config["batch_size"]

            self.dataset = dataset

            self.train_set = train_set
            self.val_set = val_set
            self.test_set = test_set
            self.best_val_loss = np.inf

            self.build()
            if self.train_set is None:
                self.setup_data([.7, .15, .15])

            self.encoder.apply(self.init_params)
            self.decoder.apply(self.init_params)

    def build(self):
        layers = []
        layer_sizes = list(sliding_window(2, self.dimensions))

        for i, size in enumerate(layer_sizes):
            layers.append(("fc"+str(i), nn.Linear(size[0], size[1])))
            if i < len(self.dimensions)-2:
                layers.append(("act"+str(i), nn.ELU()))
                layers.append(("drop"+str(i+1), nn.Dropout(self.keep_prob)))
        self.encoder = nn.Sequential(OrderedDict(layers))

        layers = []
        for i, size in enumerate(layer_sizes[-1::-1]):
            layers.append(("fc"+str(i), nn.Linear(size[1], size[0])))
            if i < len(self.dimensions)-2:
                layers.append(("act"+str(i), nn.ELU()))
                layers.append(("drop"+str(i+1), nn.Dropout(self.keep_prob)))
        self.decoder = nn.Sequential(OrderedDict(layers))

    def forward(self, x:torch.Tensor) -> torch.Tensor:
        return self.decoder(self.encoder(x))

    def training_step(self, batch, batch_idx):
        x, y = batch
        prediction = self(x)
        loss = self.loss_fn(prediction, y)

        self.log("ptl/train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch

        prediction = self(x)
        loss = self.loss_fn(prediction, y)

        self.log('ptl/val_loss', loss, prog_bar=True)
        return {"val_loss":loss}

    def test_step(self, batch, batch_idx):
        x, y = batch

        prediction = self(x)
        loss = self.loss_fn(prediction, y)

        self.log('ptl/test_loss', loss, prog_bar=True)
        return {"val_loss":loss}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        self.log("avg_val_loss", avg_loss)
        if avg_loss < self.best_val_loss:
            self.best_val_loss = avg_loss
            self.save_checkpoint()

    def save_checkpoint(self, checkpoint_dir="../../models"):
        path = os.path.join(checkpoint_dir, self.name)
        model = {"k":self.k, "dimensions":self.dimensions,"keep_prob":self.keep_prob, "name":self.name,
                 "encoder":self.encoder.state_dict(),
                 "decoder":self.decoder.state_dict()}
        if not os.path.exists(path):
            os.mkdir(path)
        with bz2.BZ2File(os.path.join(path,
                                      self.name+str(self.best_val_loss.cpu().numpy())+"."+str(self.k)+".pbz2"), "w") as f:
            pickle.dump(model, f)

    @staticmethod
    def load_checkpoint(filename, checkpoint_dir="../../models"):
        # return torch.load(os.path.join(checkpoint_dir,filename))
        with bz2.BZ2File(os.path.join(checkpoint_dir, filename), "rb") as f:
            obj = pickle.load(f)
        model = MLP(name=obj["name"], dimensions=obj["dimensions"], load=True)
        model.encoder.load_state_dict(obj["encoder"])
        model.decoder.load_state_dict(obj["decoder"])
        return model
        # self.encoder.load_state_dict(obj["encoder"])
        # self.decoder.load_state_dict(obj["decoder"])
        # self.best_val_loss = obj["val_loss"]

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.learning_rate)
        return optimizer

    def setup_data(self, split_ratio, N):
        self.n_train_samples= int(split_ratio[0]*N)
        self.n_val_samples= int(split_ratio[1] * N)
        self.n_test_samples= int(N-self.n_train_samples-self.n_val_samples)
        self.train_set, self.val_set, self.test_set = random_split(self.dataset,
                                                                   [self.n_train_samples,
                                                                    self.n_val_samples,
                                                                    self.n_test_samples])

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.batch_size, pin_memory=True)

    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=self.batch_size, pin_memory=True)

    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=self.batch_size, pin_memory=True)

    @staticmethod
    def init_params(m):
        if type(m) == nn.Linear:
            nn.init.xavier_uniform_(m.weight)
            m.bias.data.fill_(.01)

In [18]:
def train_tune(config, dimensions:list,  loss_fn=None,
                 dataset=None, train_set=None, val_set=None, test_set=None,
                 keep_prob:float=.2, num_epochs=300, num_cpus=24, num_gpus=1, model_name="model"):

    model = MLP(config, dimensions, loss_fn, dataset, train_set, val_set, test_set, keep_prob, name=model_name)
    trainer = pl.Trainer(
        max_epochs=num_epochs,
        gpus=num_gpus,
        logger=TensorBoardLogger(save_dir="logs/", name=model_name, version="0.0"),
        progress_bar_refresh_rate=20,
        callbacks=[
            TuneReportCallback({"loss":"avg_val_loss",}, on="validation_end"),
            EarlyStopping(monitor="avg_val_loss")
        ],
        precision=16,
    )
    trainer.fit(model)

def normalise(x:torch.Tensor):
    std = torch.std(x)
    std[std==0] = 1
    return (x - torch.mean(x)) / std


In [23]:
def train(datasets:list, featureList:list, config:dict,
          train_ratio:float=0.8, val_ratio:float=0.2, test_size:int=100,
          EPOCHS:int=300, hidden_dim:int=256, loss_fn=torch.nn.functional.mse_loss, keep_prob=0.2,
          n_gpu=1, n_samples=20, model_name="model",
          SEED:int=2021):
    # process data
    data = [func.processData(d, featureList) for d in datasets]
    input_data = [np.vstack(d) for d in data]
    x_tensors = [normalise(torch.from_numpy(x).float()) for x in input_data]
    y_tensors = [torch.from_numpy(x).float() for x in input_data]

    # prepare datasets
    test_sets = [(x_tensor[-test_size:], y_tensor[-test_size:]) for x_tensor, y_tensor in zip(x_tensors, y_tensors)]
    x_training = torch.vstack([x_tensor[:-test_size] for x_tensor in x_tensors])
    y_training = torch.vstack([y_tensor[:-test_size] for y_tensor in y_tensors])
    dataset = TensorDataset(x_training, y_training)
    N = len(x_training)

    train_ratio = int(train_ratio*N)
    val_ratio = int(val_ratio*N)
    print("Train: ", train_ratio, ", Validation: ", val_ratio)
    train_set, val_set = random_split(dataset, [train_ratio, val_ratio], generator=torch.Generator().manual_seed(SEED))

    # define hyperparameters
    input_dim = x_training.size()[1]
    output_dim = input_dim

    dimensions = [input_dim, hidden_dim]

    scheduler = ASHAScheduler(max_t = EPOCHS, grace_period=1, reduction_factor=2)
    reporter = CLIReporter(
        parameter_columns=["k", "lr", "batch_size"],
        metric_columns=["loss", "training_iteration"],
        max_error_rows=5,
        max_progress_rows=5,
        max_report_frequency=10)
    analysis = tune.run(
        tune.with_parameters(
            train_tune,
            dimensions = dimensions,
            loss_fn = loss_fn,
            train_set = train_set, val_set = val_set, test_set=test_sets,
            keep_prob = keep_prob,
            num_epochs = EPOCHS,
            num_gpus=n_gpu,
            model_name=model_name
        ),
        resources_per_trial= {"cpu":1, "gpu":n_gpu},
        metric="loss",
        mode="min",
        config=config,
        num_samples=n_samples,
        scheduler=scheduler,
        progress_reporter=reporter,
        name="MLP-MLP",
        verbose=False,
        checkpoint_freq=0,
        keep_checkpoints_num=1,
        checkpoint_score_attr="loss",
        checkpoint_at_end=True
    )
    print("Best hyperparameters found were: ", analysis.best_config)
    print("Best achieved loss was: ", analysis.best_result)
    return test_sets

def clean_checkpoints(num_keep=3, path="../../models"):
    for dir, dname, files in os.walk(path):
        saved_checkpoints = []
        for fname in files:
            fname = fname.split(".")
            saved_checkpoints.append(fname)
        print("Num checkpoints in {}: {}".format(dir, len(saved_checkpoints)))

        saved_checkpoints.sort(key = lambda x: x[1])
        for filename in saved_checkpoints[3:]:
            os.remove(os.path.join(dir,".".join(filename)))

def test(model:torch.nn.Module, test_sets:list, loss_fn, set_names=list,
         save=True, path="../../results", model_name="model"):
    # Intra test performance
    df = {}
    for i,t1 in enumerate(test_sets):
        for j, t2 in enumerate(test_sets):
            x = t1[0]
            y = t2[1]
            pred = model(x)
            loss = loss_fn(x, y)
            df["{}-{}".format(set_names[i], set_names[j])] = [loss.cpu().numpy()]
            print("Test encoding {} to {}, MSE={:.2f}".format(set_names[i], set_names[j], loss))
    filepath = os.path.join(path, model_name)
    if not os.path.exists(filepath): os.mkdir(filepath)
    pd.DataFrame(df).to_csv(os.path.join(filepath, "tests.csv"))

# Testing pos, rotMat, velocity

In [5]:
# Prepare train data
data_path = "../../data/"

# load data
data_1 = func.load(data_path+"LOCO_R2-default-locomotion.pbz2")
data_2 = func.load(data_path+"LOCO_R2-default-locomotion-small.pbz2")
data_3 = func.load(data_path+"LOCO_R2-default-locomotion-large.pbz2")

In [6]:
config = {
    "k":tune.randint(2, 256),
    "lr": tune.loguniform(1e-3, 1e-7),
    "batch_size":tune.choice([12, 24, 48])
}

test_sets = train([data_1, data_2, data_3], ["pos", "rotMat", "velocity"],
                  config, n_samples=30, model_name="MLP-MLP")

2021-03-30 21:44:21,066	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265
2021-03-30 21:44:25,230	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265
2021-03-30 21:44:29,297	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265
2021-03-30 21:44:32,867	INFO ray_trial_executor.py:197 -- Initializing Ray automatically.For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run`.
2021-03-30 21:44:33,260	INFO services.py:1172 -- View the Ray dashboard at http://127.0.0.1:8265
2021-03-30 21:44:34,614	WARNING function_runner.py:540 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.
(pid=1892880) GPU available: True, used: True
(pid=1892880) TPU available: None, using: 0 TPU cores
(pid=1892880) Using native 16bit precision.
(pid=1892880) 

Train:  3216 , Validation:  804
Epoch 0:  71%|███████▏  | 120/168 [00:00<00:00, 438.73it/s, loss=0.217, v_num=0.0, ptl/val_loss=0.859]
Validating: 0it [00:00, ?it/s]
Epoch 0:  71%|███████▏  | 120/168 [00:00<00:00, 438.73it/s, loss=0.217, v_num=0.0, ptl/val_loss=0.859]
Validating: 0it [00:00, ?it/s]
Epoch 0:  71%|███████▏  | 120/168 [00:00<00:00, 438.73it/s, loss=0.217, v_num=0.0, ptl/val_loss=0.859]
Validating: 0it [00:00, ?it/s]
Epoch 0:  71%|███████▏  | 120/168 [00:00<00:00, 438.73it/s, loss=0.217, v_num=0.0, ptl/val_loss=0.859]
Validating: 0it [00:00, ?it/s]
Epoch 1:  71%|███████▏  | 120/168 [00:00<00:00, 436.17it/s, loss=0.117, v_num=0.0, ptl/val_loss=0.0907]
Validating: 0it [00:00, ?it/s]
Epoch 1:  71%|███████▏  | 120/168 [00:00<00:00, 436.17it/s, loss=0.117, v_num=0.0, ptl/val_loss=0.0907]
Validating: 0it [00:00, ?it/s]
Epoch 1:  71%|███████▏  | 120/168 [00:00<00:00, 436.17it/s, loss=0.117, v_num=0.0, ptl/val_loss=0.0907]
Validating: 0it [00:00, ?it/s]
Epoch 1:  71%|███████▏  | 1

In [24]:
# clean_checkpoints()

best_model = MLP.load_checkpoint(filename="/home/nuoc/checkpoints/MLP-MLP/MLP-MLP0.0016448386.92.pbz2")
test(best_model, test_sets, nn.functional.mse_loss, ["default", "small", "large"])


Test encoding default to default, MSE=0.23
Test encoding default to small, MSE=0.53
Test encoding default to large, MSE=0.30
Test encoding small to default, MSE=0.61
Test encoding small to small, MSE=0.17
Test encoding small to large, MSE=0.73
Test encoding large to default, MSE=0.32
Test encoding large to small, MSE=0.68
Test encoding large to large, MSE=0.27


# Testing pos, rotMat

In [ ]:
# Prepare train data
data_path = "../../data/"
# load data
data_1 = func.load(data_path+"LOCO_R2-default-locomotion.pbz2")
data_2 = func.load(data_path+"LOCO_R2-default-locomotion-small.pbz2")
data_3 = func.load(data_path+"LOCO_R2-default-locomotion-large.pbz2")

In [ ]:
config = {
    "k":tune.randint(2, 256),
    "lr": tune.loguniform(1e-3, 1e-7),
    "batch_size":tune.choice([12, 24, 48])
}

test_sets = train([data_1, data_2, data_3], ["pos", "rotMat", ],
                  config, n_samples=20, model_name="MLP-MLP_POS_ROT")

In [10]:
best_model = clean_checkpoints()
# best_model = MLP.load_checkpoint(filename=best_model)
# test(best_model, test_sets, nn.functional.mse_loss, ["default", "small", "large"])

dir:  /home/nuoc/checkpoints
Num checkpoints in ['MLP-MLP', 'yo']: 0
dir:  /home/nuoc/checkpoints/MLP-MLP
Num checkpoints in []: 142
dir:  /home/nuoc/checkpoints/yo
Num checkpoints in []: 1


IndexError: list index out of range

# Testing pos, rotMat, velocity, angularVelocity,

In [ ]:
# Prepare train data
data_path = "../../data/"
# load data
data_1 = func.load(data_path+"LOCO_R2-default-locomotion.pbz2")
data_2 = func.load(data_path+"LOCO_R2-default-locomotion-small.pbz2")
data_3 = func.load(data_path+"LOCO_R2-default-locomotion-large.pbz2")

In [ ]:
config = {
    "k":tune.randint(2, 256),
    "lr": tune.loguniform(1e-3, 1e-7),
    "batch_size":tune.choice([12, 24, 48])
}

test_sets = train([data_1, data_2, data_3], ["pos", "rotMat", "velocity", "angularVelocity"],
                  config, n_samples=20, model_name="MLP-MLP_POS_ROT_VEL_AVEL")

In [ ]:
best_model = clean_checkpoints()
best_model = MLP.load_checkpoint(filename=best_model)
test(best_model, test_sets, nn.functional.mse_loss, ["default", "small", "large"])

In [ ]:
# Prepare train data
data_path = "../../data/"
# load data
data_1 = func.load(data_path+"LOCO_R2-default-locomotion.pbz2")
data_2 = func.load(data_path+"LOCO_R2-default-locomotion-small.pbz2")
data_3 = func.load(data_path+"LOCO_R2-default-locomotion-large.pbz2")

In [ ]:
config = {
    "k":tune.randint(2, 256),
    "lr": tune.loguniform(1e-3, 1e-7),
    "batch_size":tune.choice([12, 24, 48])
}

test_sets = train([data_1, data_2, data_3], ["currentValue", "targetValue"],
                  config, n_samples=20, model_name="MLP-MLP_cValue,tValue")

In [ ]:
best_model = clean_checkpoints()
best_model = MLP.load_checkpoint(filename=best_model)
test(best_model, test_sets, nn.functional.mse_loss, ["default", "small", "large"])